In [1]:
import sys 
sys.path.append(rf"/nfs/home/genovese/thesis-wildfire-genovese/src")
from importlib import reload
import utils
reload(utils)
from utils import *
data_folder = "/nfs/home/genovese/thesis-wildfire-genovese/data/"
output_folder = "/nfs/home/genovese/thesis-wildfire-genovese/data/gathering/per_comune/"

In [ ]:
ist = pd.read_excel('/nfs/home/genovese/thesis-wildfire-genovese/data/raw/istruzione_occupazione/istruzione_2018-2022.xlsx')
occ = pd.read_excel('/nfs/home/genovese/thesis-wildfire-genovese/data/raw/istruzione_occupazione/occupazione_2018_2022.xlsx')

In [2]:
confini = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/clean_data/confini_piemonte/confini_piemonte.shp')

---

In [ ]:
istruzione = dict.fromkeys([2018, 2019, 2020, 2021, 2022])
for i, a in enumerate(istruzione.keys()):
    istruzione[a] = ist.iloc[:-1, [0]+list(range(1+(11*i), 12+(11*i)))]

In [ ]:
for a, df in istruzione.items():
    df.columns = df.iloc[0, :]
    df.drop(index=df.index[:2], inplace=True)

Remove data gathered at region/province level

In [ ]:
for a, df in istruzione.items():
    to_drop = list(df[df['Grado di istruzione'].str.strip().duplicated(keep='last')].index) + list(df[df['Grado di istruzione'].str.strip() == 'Piemonte'].index)
    istruzione[a] = df.drop(index = to_drop).reset_index(drop=True)

In [ ]:
for a,df in istruzione.items():
    aux = df.apply(lambda x: uniform_strings(x, 'Grado di istruzione'), axis=1).replace('..', -1000).fillna(-1000)
    for c in aux.columns[1:]:
        aux.loc[:, c] = aux[c].astype('int64')
        aux.loc[:, c] = aux[c].replace(-1000, None)
    istruzione[a] = aux

In [ ]:
for a, df in istruzione.items():
    istruzione[a] = confini[['comune', 'geometry']].merge(df.rename(columns={'Grado di istruzione': 'comune'}), on='comune', how='inner')

In [ ]:
for a, df in istruzione.items():    
    save_clean_data(df.drop(columns='comune'), str(a), '/nfs/home/genovese/thesis-wildfire-genovese/data/gathering/per_comune/istruzione')

---

In [ ]:
occupazione = dict.fromkeys([2018, 2019, 2020, 2021, 2022])
for i, a in enumerate(occupazione.keys()):
    occupazione[a] = occ.iloc[1:-1, [0]+list(range(1+(9*i), 10+(9*i)))]

In [ ]:
for a, df in occupazione.items():
    df.columns = df.iloc[0, :]
    df.drop(index=df.index[:2], inplace=True)

In [ ]:
for a, df in occupazione.items():
    to_drop = list(df[df['Condizione professionale'].str.strip().duplicated(keep='last')].index) + list(df[df['Condizione professionale'].str.strip() == 'Piemonte'].index)
    occupazione[a] = df.drop(index = to_drop).reset_index(drop=True)

In [ ]:
for a,df in occupazione.items():
    aux = df.apply(lambda x: uniform_strings(x, 'Condizione professionale'), axis=1).replace('..', -1000).fillna(-1000)
    for c in aux.columns[1:]:
        aux.loc[:, c] = aux[c].astype('int64')
        aux.loc[:, c] = aux[c].replace(-1000, None)
    occupazione[a] = aux

In [ ]:
for a, df in occupazione.items():
    occupazione[a] = confini[['comune', 'geometry']].merge(df.rename(columns={'Condizione professionale': 'comune'}), on='comune', how='inner')

In [ ]:
for a, df in occupazione.items():    
    save_clean_data(df.drop(columns='comune'), str(a), '/nfs/home/genovese/thesis-wildfire-genovese/data/gathering/per_comune/occupazione')

---

In [124]:
def istruzione_dfs(ist):
    istruzione = dict.fromkeys([2018, 2019, 2020, 2021, 2022])
    while(str(ist.iloc[0, 0]).strip() != 'Grado di istruzione'):
        ist.drop(index=ist.index[0], inplace=True)
    row = ist.iloc[0, :]
    pos = list(row.index).index(row.index[row.eq('totale')][0])
    for i, a in enumerate(istruzione.keys()):
        istruzione[a] = ist.iloc[:-1, [0]+list(range(1+(pos*i), (pos+1)+(pos*i)))]
    
    for a, df in istruzione.items():
        while(pd.isna(df.iloc[0, 0])):
            df.drop(index=df.index[0], inplace=True)
        df.columns = df.iloc[0, :]
        df.drop(index=df.index[0], inplace=True)
        while(pd.isna(df.iloc[0, 0])):
            df.drop(index=df.index[0], inplace=True)
        to_drop = list(df[df['Grado di istruzione'].str.strip().duplicated(keep='last')].index) + list(df[df['Grado di istruzione'].str.strip() == 'Piemonte'].index)
        aux = df.drop(index = to_drop).reset_index(drop=True)
        aux = df.apply(lambda x: uniform_strings(x, 'Grado di istruzione'), axis=1).replace('..', -1000).fillna(-1000)
        for c in aux.columns[1:]:
            aux.loc[:, c] = aux[c].astype('int64')
            aux.loc[:, c] = aux[c].replace(-1000, None)
        
        istruzione[a] = confini[['comune', 'geometry']].merge(aux.rename(columns={'Grado di istruzione': 'comune'}), on='comune', how='inner')
    return istruzione

In [125]:
def occupazione_dfs(occ):
    occupazione = dict.fromkeys([2018, 2019, 2020, 2021, 2022])
    while(str(occ.iloc[0, 0]).strip() != 'Condizione professionale'):
        occ.drop(index=occ.index[0], inplace=True)
    row = occ.iloc[0, :]
    pos = list(row.index).index(row.index[row.eq('totale')][0])
    for i, a in enumerate(occupazione.keys()):
        occupazione[a] = occ.iloc[:-1, [0]+list(range(1+(pos*i), (pos+1)+(pos*i)))]

    for a, df in occupazione.items():
        while(pd.isna(df.iloc[0, 0])):
            df.drop(index=df.index[0], inplace=True)
        df.columns = df.iloc[0, :]
        df.drop(index=df.index[0], inplace=True)
        while(pd.isna(df.iloc[0, 0])):
            df.drop(index=df.index[0], inplace=True)
        to_drop = list(df[df['Condizione professionale'].str.strip().duplicated(keep='last')].index) + list(df[df['Condizione professionale'].str.strip() == 'Piemonte'].index)
        aux = df.drop(index = to_drop).reset_index(drop=True)
        aux = df.apply(lambda x: uniform_strings(x, 'Condizione professionale'), axis=1).replace('..', -1000).fillna(-1000)
        for c in aux.columns[1:]:
            aux.loc[:, c] = aux[c].astype('int64')
            aux.loc[:, c] = aux[c].replace(-1000, None)
        
        occupazione[a] = confini[['comune', 'geometry']].merge(aux.rename(columns={'Condizione professionale': 'comune'}), on='comune', how='inner')
    return occupazione

---

In [ ]:
occ_15_24 = occupazione_dfs(pd.read_excel('/nfs/home/genovese/thesis-wildfire-genovese/data/raw/istruzione_occupazione/occupazione_15_24.xlsx'))
occ_25_49 = occupazione_dfs(pd.read_excel('/nfs/home/genovese/thesis-wildfire-genovese/data/raw/istruzione_occupazione/occupazione_25_49.xlsx').drop(columns=pd.read_excel('/nfs/home/genovese/thesis-wildfire-genovese/data/raw/istruzione_occupazione/occupazione_25_49.xlsx').columns[1]))
occ_50_64 = occupazione_dfs(pd.read_excel('/nfs/home/genovese/thesis-wildfire-genovese/data/raw/istruzione_occupazione/occupazione_50_64.xlsx'))
occ_65 = occupazione_dfs(pd.read_excel('/nfs/home/genovese/thesis-wildfire-genovese/data/raw/istruzione_occupazione/occupazione_65.xlsx').drop(columns=pd.read_excel('/nfs/home/genovese/thesis-wildfire-genovese/data/raw/istruzione_occupazione/occupazione_65.xlsx').columns[1]))

In [ ]:
ist_9_24 = istruzione_dfs(pd.read_excel('/nfs/home/genovese/thesis-wildfire-genovese/data/raw/istruzione_occupazione/istruzione_9_24.xlsx'))
ist_25_49 = istruzione_dfs(pd.read_excel('/nfs/home/genovese/thesis-wildfire-genovese/data/raw/istruzione_occupazione/istruzione_25_49.xlsx'))
ist_50_64 = istruzione_dfs(pd.read_excel('/nfs/home/genovese/thesis-wildfire-genovese/data/raw/istruzione_occupazione/istruzione_50_64.xlsx'))
ist_65 = istruzione_dfs(pd.read_excel('/nfs/home/genovese/thesis-wildfire-genovese/data/raw/istruzione_occupazione/istruzione_65.xlsx'))

In [165]:
occupazione = dict.fromkeys([2018, 2019, 2020, 2021, 2022])
istruzione = dict.fromkeys([2018, 2019, 2020, 2021, 2022])

for a in occupazione.keys():
    occupazione[a] = gpd.read_file(f'/nfs/home/genovese/thesis-wildfire-genovese/data/gathering/per_comune/occupazione/{a}/{a}.shp').drop(columns=gpd.read_file(f'/nfs/home/genovese/thesis-wildfire-genovese/data/gathering/per_comune/occupazione/{a}/{a}.shp').columns[0])


for a in istruzione.keys():
    istruzione[a] = gpd.read_file(f'/nfs/home/genovese/thesis-wildfire-genovese/data/gathering/per_comune/istruzione/{a}/{a}.shp').drop(columns=gpd.read_file(f'/nfs/home/genovese/thesis-wildfire-genovese/data/gathering/per_comune/istruzione/{a}/{a}.shp').columns[0])

In [166]:
for a in [2018, 2019, 2020, 2021, 2022]:
    occ_15_24[a].columns = ['comune', 'geometry'] + ['15_24_' + s for s in list(occ_15_24[a].columns[2:])]
    occ_25_49[a].columns = ['comune', 'geometry'] + ['25_49_' + s for s in list(occ_25_49[a].columns[2:])]
    occ_50_64[a].columns = ['comune', 'geometry'] + ['50_64_' + s for s in list(occ_50_64[a].columns[2:])]
    occ_65[a].columns = ['comune', 'geometry'] + ['65_' + s for s in list(occ_65[a].columns[2:])]

    ist_9_24[a].columns = ['comune', 'geometry'] + ['9_24_' + s for s in list(ist_9_24[a].columns[2:])]
    ist_25_49[a].columns = ['comune', 'geometry'] + ['25_49_' + s for s in list(ist_25_49[a].columns[2:])]
    ist_50_64[a].columns = ['comune', 'geometry'] + ['50_64_' + s for s in list(ist_50_64[a].columns[2:])]
    ist_65[a].columns = ['comune', 'geometry'] + ['65_' + s for s in list(ist_65[a].columns[2:])]

In [167]:
ISTRUZIONE = dict()
OCCUPAZIONE = dict()

for a in [2018, 2019, 2020, 2021, 2022]:
    ISTRUZIONE[a] = istruzione[a].merge(ist_9_24[a], on='geometry', how='inner').drop(columns='comune')
    ISTRUZIONE[a] = ISTRUZIONE[a].merge(ist_25_49[a], on='geometry', how='inner').drop(columns='comune')
    ISTRUZIONE[a] = ISTRUZIONE[a].merge(ist_50_64[a], on='geometry', how='inner').drop(columns='comune')
    ISTRUZIONE[a] = ISTRUZIONE[a].merge(ist_65[a], on='geometry', how='inner').drop(columns='comune')
    
    OCCUPAZIONE[a] = occupazione[a].merge(occ_15_24[a], on='geometry', how='inner').drop(columns='comune')
    OCCUPAZIONE[a] = OCCUPAZIONE[a].merge(occ_25_49[a], on='geometry', how='inner').drop(columns='comune')
    OCCUPAZIONE[a] = OCCUPAZIONE[a].merge(occ_50_64[a], on='geometry', how='inner').drop(columns='comune')
    OCCUPAZIONE[a] = OCCUPAZIONE[a].merge(occ_65[a], on='geometry', how='inner').drop(columns='comune')

In [ ]:
for a, df in OCCUPAZIONE.items():
    save_clean_data(df, str(a), '/nfs/home/genovese/thesis-wildfire-genovese/data/gathering/per_comune/OCCUPAZIONE')

for a, df in ISTRUZIONE.items():
    save_clean_data(df, str(a), '/nfs/home/genovese/thesis-wildfire-genovese/data/gathering/per_comune/ISTRUZIONE')

---